
## Table of Contents
- [Introduction](#intro)
- [Part I - Data Cleaning](#Cleaning)
- [Part II - Probability](#probability)
- [Part III - A/B Test](#ab_test)
- [Part IV - Regression](#regression)






<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that we get some practice working with the difficulties of these.

For this project, we will be working to understand the results of an A/B test run by an e-commerce website.  Our goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.



<a id='Cleaning'></a>
#### Part I - Data Cleaning 

To get started, let's import our libraries.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt # basic visualizations 
import seaborn as sns # advanced visualizations

import random
random.seed(42) #We are setting the seed to assure you get the same answers

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#read the data and display the top 5 rows
df = pd.read_csv("../input/ecommerce-ab-testing/ab_test.csv")
df.head()

In [ ]:
# change column names 
df.columns = ["user_id", "timestamp", "group", "landing_page", "converted"]
df.head()

In [ ]:
#numer of rows and unique users
print(f'Number of rows: {df.shape[0]}')
print(f'Number of unique users: {df.user_id.nunique()}')

In [ ]:
#general info
df.info()

In [ ]:
#missing values
df.isna().sum()

In [ ]:
#Does the number of new_page and treatment match?
n_treat = df[df["group"] == "treatment"].shape[0]
n_new_page = df[df["landing_page"] == "new_page"].shape[0]
difference = n_treat - n_new_page

pd.DataFrame({
    'N treatment': [n_treat],
    'N new_page': [n_new_page],
    'Difference': [difference]
})

There is mismatch between number of users assigned to treatment and the number of those landed on treatment page. This might indicate a problem with the data and needs further exploration.

In [ ]:
# lets see those rows 
df[(df["group"] == "treatment") & (df["landing_page"] == "old_page")]

In [ ]:
df_mismatch = df[(df["group"] == "treatment") & (df["landing_page"] == "old_page")
               |(df["group"] == "control") & (df["landing_page"] == "new_page")]

n_mismatch = df_mismatch.shape[0]

percent_mismatch = round(n_mismatch / len(df) * 100, 2)
print(f'Number of mismatched rows: {n_mismatch} rows')
print(f'Percent of mismatched rows: {percent_mismatch} percent')

As you can see, there are 3893 rows where treatment does not match with new_page or control does not match with old_page, we cannot be sure if this row truly received the new or old page. How we will deal with those rows?

In [ ]:
df2 = df[(df["group"] == "treatment") & (df["landing_page"] == "new_page")
        |(df["group"] == "control") & (df["landing_page"] == "old_page")]

len(df2)

In [ ]:
df2.head()

In [ ]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

In [ ]:
# Another way to double Check all of the correct rows were removed 
df_mismatch = df2[(df2["group"] == "treatment") & (df2["landing_page"] == "old_page")
               |(df2["group"] == "control") & (df2["landing_page"] == "new_page")]

n_mismatch = df_mismatch.shape[0]

percent_mismatch = round(n_mismatch / len(df2) * 100, 2)
print(f'Number of mismatched rows: {n_mismatch} rows')
print(f'Percent of mismatched rows: {percent_mismatch} percent')

In [ ]:
# unique user id in df2 
df2.user_id.nunique()

In [ ]:
# number of repeated ids in df2
len(df2) - df2.user_id.nunique()

In [ ]:
# Display the duplicated row 
df2[df2.duplicated("user_id") == True]

In [ ]:
#drop the duplicated row
df2 = df2.drop_duplicates("user_id") 

In [ ]:
# Douple Check that it is actually dropped
len(df2) - df2.user_id.nunique()

<a id='probability'></a>
#### Part II - Probability

In [ ]:
# Percent of convergance
# The probability of an individual converting regardless of the page they receive
df2.converted.mean() * 100

In [ ]:
#Given that an individual was in the control group, what is the probability they converted?
#Given that an individual was in the treatment group, what is the probability they converted?
df2.user_id = df2.user_id.astype(str)
df2.groupby("group").mean() * 100

In [ ]:
#What is the probability that an individual received the new page?
pd.DataFrame(df2.landing_page.value_counts(normalize = True) * 100)

#### Is there a sufficient evidence to conclude that the new treatment page leads to more conversions?

1. The probability that an individual received the new page is 50%
2. The probability of an individual converting regardless of the page they receive is 11.96%
3. Given that an individual was in the control group, the probability they converted is 12.04%
4. Given that an individual was in the treatment group, the probability they converted is 11.88%

1 to 4 suggests that there is no significant difference in convergence between treatment and control groups. Therefore we may conclude that the new treatment page has no impact and does not lead to more conversions.

<a id='ab_test'></a>
### Part III - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time? How long do you run to render a decision that neither page is better than another?

These questions are the difficult parts associated with A/B tests in general. 

For now, consider you need to make the decision just based on all the data provided. If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%

**$H_{0}$**: **$p_{old}$** - **$p_{new}$** **$>= 0$**

**$H_{1}$**: **$p_{old}$** - **$p_{new}$** **$ < 0$**

In [ ]:
# Creating the sampling distribution of difference in means 
means_diff = []
size = df.shape[0]
for _ in range(10000):
    sample = df2.sample(size, replace = True)
    control_mean = sample[sample["group"] == "control"]["converted"].mean()
    treat_mean = sample[sample["group"] == "treatment"]["converted"].mean()
    means_diff.append(treat_mean - control_mean)

In [ ]:
# Plotting the sampling distribution 
plt.figure(figsize = (8,4), dpi = 100)
plt.hist(means_diff, bins = 25)
plt.show()

In [ ]:
# Simulate distribution under the null hypothesis
means_diff = np.array(means_diff)
null_vals = np.random.normal(0, means_diff.std(), means_diff.size)

In [ ]:
# Plot the null distribution
plt.figure(figsize = (8,4), dpi = 100)
plt.hist(null_vals, bins = 25)
plt.show()

In [ ]:
# Plot observed statistic with the null distibution
control_mean = df2[df2["group"] == "control"]["converted"].mean()
treat_mean = df2[df2["group"] == "treatment"]["converted"].mean()
obs_diff = treat_mean - control_mean

plt.figure(figsize = (8,4), dpi = 100)
plt.hist(null_vals, bins = 25)
plt.axvline(obs_diff, c='red')
plt.show()

In [ ]:
# calculating the p value 
(null_vals > obs_diff).mean()

#### Concluding remarks for this section so far:

1. The p_value (0.9) is greater than alpha, therefore we fail to reject the null.
2. This emphasizes of initial conclusion that there is no significant impact for the new page.

We could also use a built-in to achieve similar results. Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance.

In [ ]:
import statsmodels.api as sm

convert_old = df2[(df2["converted"] == 1) & (df2["landing_page"] == "old_page")]['user_id'].nunique()
convert_new = df2[(df2["converted"] == 1) & (df2["landing_page"] == "new_page")]['user_id'].nunique()
n_old = df2[df["landing_page"] == "old_page"]['user_id'].nunique()
n_new = df2[df["landing_page"] == "new_page"]['user_id'].nunique()

In [ ]:
#Compute test statistic and p-value
z_score, p_value = sm.stats.proportions_ztest(np.array([convert_new,convert_old]),np.array([n_new,n_old]), alternative = 'larger')

In [ ]:
# Print Z Score and P_Value
z_score, p_value 

Using test statistic and p-value, we reach the same coclusion: we can not reject the null 

<a id='regression'></a>
### Part IV - Regression

In this final part, you will see that the result we achieved in the A/B test Part above can also be achieved by performing regression. Since each row is either a conversion or no conversion, we will use logestic regression to see if there is a significant difference in conversion based on which page a customer receives. However, we first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.

In [ ]:
# Creat the intercept 
df2["intercept"] = 1
df2.head()

In [ ]:
# Create ab_page column, which is 1 when an individual receives the treatment and 0 if control.
df2["ab_page"] = df2.group.apply(lambda x: 1 if (x == "treatment") else 0)
df2.head()

In [ ]:
# Instantiate and fit the regression model
model = sm.Logit(df2['converted'], df2[['intercept','ab_page']])
result = model.fit()
result.summary()

The P-Value is 0.190, It is different from the one we obtained from the previous analysis because the null hypothesis is different in both cases.

We might add additional features to our model such timestamp, the reason for that conversion might differ according to the time at which the user visits the website. But this might come with a disadvantage, the model will become more complicated and less interpretable. It might be also  susceptible to overfitting.


We will leave timestamp for now. Instead, along with testing if the conversion rate changes for different pages, we will also add an effect based on which country a user lives in. we will need to read in the countries.csv dataset and merge together your datasets on the appropriate rows.

In [ ]:
# Read the country data
countries = pd.read_csv("../input/ecommerce-ab-testing/countries_ab.csv")
countries.head()

In [ ]:
# Merge the countries dataframe with df2 
countries.columns = ["user_id", "country"]
countries["user_id"] = countries["user_id"].astype(str)
df3 = df2.merge(countries, on = "user_id", how = "left")
df3.head()

In [ ]:
# creating dummies for country and landing_page columns 
df3[['CA','UK','US']] = pd.get_dummies(df3['country'])
df3[['new_page','old_page']] = pd.get_dummies(df3['landing_page'])
df3.head()

In [ ]:
# lest see if there is a relation between country and conversion
pd.pivot_table(data = df3, index = "country", values = "converted").sort_values(by = "converted", ascending = False) * 100

It seems that coutry has a very minor impact on convergance. we will see its true impact along with ather features in the regression# Instantiate and fit the regression model
model = sm.Logit(df2['converted'], df2[['intercept','ab_page']])
result = model.fit()
result.summary()# Instantiate and fit the regression model
model = sm.Logit(df2['converted'], df2[['intercept','ab_page']])
result = model.fit()
result.summary()

In [ ]:
# Instantiate and fit the regression model with country as an additional variable: 'CA' is a baseline
model = sm.Logit(df3['converted'], df3[['intercept','ab_page', 'UK','US']])
result = model.fit()
result.summary()

In [ ]:
# exponentiate the parameters to inteprete the result
np.exp(result.params)

All the coefficients are statistically insignificant except the intercept. This comes inline with the initial conclusions that we have just made.

#### Summary:
In this notebook we conducted a detailed A/B testing using 3 main methods:
1. Sampling distribution
2. Z test
3. Logestic regression 

All three methods resulted in the same conclusion: the treatment has no impact.